# Pedestrian classification
Erik Matovič and Jakub Horvat

[Dataset](https://data.nvision2.eecs.yorku.ca/PIE_dataset/)

### 0. Imports

In [36]:
import pandas as pd
from matplotlib import pyplot as plt
from os import listdir
from os.path import join
from pie_data import PIE
import cv2
from math import floor, ceil

In [37]:
pie_path = '../' #'/media/erik/T7/DP/PIE_data' # <path_to_the_root_folder>
pie = PIE(data_path=pie_path)
type(pie)

pie_data.PIE

In [3]:
train_list = pie._get_image_set_ids('train')
test_list = pie._get_image_set_ids('test')
val_list = pie._get_image_set_ids('val')

train_list, val_list, test_list

(['set01', 'set02', 'set04'], ['set05', 'set06'], ['set03'])

In [40]:
class PIE_peds():
    def __init__(self, setname: str, pie: PIE) -> None:
        #super().__init__()
        self.setname = setname
        self.set_dir = dict()
        self.all_path = list()
        self.all_filenames = list()
        self.path = pie._images_path

        assert setname in ['train', 'val', 'test']
        
        self.img_init(pie)
        
    
    def __len__(self):
        return len(self.all_filenames)
    

    #def __getitem__(self, index):
    #    pass
        #return super().__getitem__(index)


    def img_init(self, pie) -> None:
        set_list = pie._get_image_set_ids(self.setname)

        for set_ in set_list:
            set_path = join(pie._images_path, set_)
            self.set_dir[set_] = dict()
            
            set_content = listdir(set_path)
            for video in set_content:
                video_path = join(set_path, video)
                imgs = listdir(video_path)
                self.all_path += ([video_path] * len(imgs))
                self.all_filenames += imgs
                self.set_dir[set_][video] = imgs
        print(self.set_dir)    


In [42]:
train_dataset = PIE_peds('train', pie)
val_dataset = PIE_peds('val', pie)
test_dataset = PIE_peds('test', pie)

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

{'set01': {'video_0004': ['00002.png', '00005.png', '00001.png', '00000.png', '00013.png', '00003.png', '00006.png', '00007.png', '00008.png', '00015.png', '00010.png', '00004.png', '00011.png', '00014.png', '00009.png', '00012.png'], 'video_0001': ['01127.png', '01130.png', '01134.png', '01129.png', '01139.png', '01137.png', '01132.png', '01128.png', '01131.png', '01133.png', '01140.png', '01138.png', '01125.png', '01135.png', '01136.png', '01126.png'], 'video_0003': ['00127.png', '00128.png', '00121.png', '00134.png', '00131.png', '00123.png', '00135.png', '00129.png', '00130.png', '00132.png', '00124.png', '00126.png', '00122.png', '00125.png', '00133.png', '00120.png'], 'video_0002': ['04698.png', '04697.png', '04702.png', '04704.png', '04709.png', '04707.png', '04705.png', '04710.png', '04695.png', '04699.png', '04696.png', '04703.png', '04701.png', '04706.png', '04700.png', '04708.png']}, 'set02': {'video_0001': ['02830.png', '02836.png', '02838.png', '02825.png', '02823.png', '0

In [45]:
def save_img_annotations(dataset: PIE_peds, folder: str) -> None:
    """
    Save images into given folder and also CSV annotations. 

    :param dataset: PIE_peds class, where all paths are set. 
    :param folder: folder train or val or test to save pedestrians.
    """

    assert folder in ['train', 'val', 'test']

    target = dict()
    target['set'] = []
    target['video'] = []
    target['frame'] = []
    target['ped_idx'] = []
    target['ped'] = []
    target['BBox'] = []
    target['action'] = []
    target['gesture'] = []
    target['look'] = []
    target['cross'] = []

    for set_name in dataset.set_dir:
        for video in dataset.set_dir[set_name]:
            annotations = pie._get_annotations(set_name, video)
            for frame in dataset.set_dir[set_name][video]: 
                img_path = '../images/' + set_name + '/' + video + '/' + frame
                #print(img_path, type(img_path))

                img = cv2.imread(img_path)
                # plt.imshow(img)
                for idx in annotations['ped_annotations']:
                    frame_idx = int(frame[:-4])
                    #print(idx)
                    if frame_idx in annotations['ped_annotations'][idx]['frames']:
                        frame_key = annotations['ped_annotations'][idx]['frames'].index(frame_idx)
                        
                        # set annotations in dictionary
                        target['set'].append(set_name)
                        target['video'].append(video)
                        target['frame'].append(frame)
                        target['ped_idx'].append(frame_key)
                        target['ped'].append(idx)
                        target['BBox'].append(annotations['ped_annotations'][idx]['bbox'][frame_key])
                        target['action'].append(annotations['ped_annotations'][idx]['behavior']['action'][frame_key])
                        target['gesture'].append(annotations['ped_annotations'][idx]['behavior']['gesture'][frame_key])
                        target['look'].append(annotations['ped_annotations'][idx]['behavior']['look'][frame_key])
                        target['cross'].append(annotations['ped_annotations'][idx]['behavior']['cross'][frame_key])
                        
                        # BBox for pedestrian
                        x1 = floor(target['BBox'][-1][0])
                        y1 = floor(target['BBox'][-1][1])
                        x2 = ceil(target['BBox'][-1][2])
                        y2 = ceil(target['BBox'][-1][3])
                        crop_img = img[y1:y2, x1:x2]
                        crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
                        crop_img = cv2.resize(crop_img, (64, 64))
                        
                        f_name = '../images/' + folder + '/' + set_name + '_' + video + '_' + target['ped'][-1] + '_' + frame
                        #print(f_name)
                        cv2.imwrite(f_name, crop_img)

                        #cv2.imshow("cropped", crop_img)
                        #cv2.waitKey(0)
                        #cv2.destroyAllWindows()
                        #print(type(target['BBox'][-1][1]))
                        #break
    df = pd.DataFrame(data=target)
    annotations_path = '../' + folder + '_annotations.csv'
    df.to_csv(annotations_path)
    print(f'{folder} done!')
    #return target


save_img_annotations(train_dataset, 'train')
save_img_annotations(val_dataset, 'val')
save_img_annotations(test_dataset, 'test')

../annotations/set01/video_0004_annt.xml
../annotations/set01/video_0001_annt.xml
../annotations/set01/video_0003_annt.xml
../annotations/set01/video_0002_annt.xml
../annotations/set02/video_0001_annt.xml
../annotations/set02/video_0003_annt.xml
../annotations/set02/video_0002_annt.xml
../annotations/set04/video_0005_annt.xml
../annotations/set04/video_0012_annt.xml
../annotations/set04/video_0011_annt.xml
../annotations/set04/video_0009_annt.xml
../annotations/set04/video_0015_annt.xml
../annotations/set04/video_0016_annt.xml
../annotations/set04/video_0014_annt.xml
../annotations/set04/video_0004_annt.xml
../annotations/set04/video_0001_annt.xml
../annotations/set04/video_0008_annt.xml
../annotations/set04/video_0006_annt.xml
../annotations/set04/video_0003_annt.xml
../annotations/set04/video_0010_annt.xml
../annotations/set04/video_0007_annt.xml
../annotations/set04/video_0002_annt.xml
../annotations/set04/video_0013_annt.xml
train done!
../annotations/set05/video_0001_annt.xml
../a

In [48]:
df_train = pd.read_csv('../train_annotations.csv')
df_train

,Unnamed: 0,set,video,frame,ped_idx,ped,BBox,action,gesture,look,cross
0,0,set01,video_0004,00002.png,2,1_4_112,"[1075.3, 802.91, 1084.38, 828.83]",1,0,0,0
1,1,set01,video_0004,00005.png,5,1_4_112,"[1078.43, 803.14, 1088.09, 829.76]",1,0,0,0
2,2,set01,video_0004,00001.png,1,1_4_112,"[1074.26, 802.84, 1083.14, 828.51]",1,0,0,0
3,3,set01,video_0004,00000.png,0,1_4_112,"[1073.21, 802.76, 1081.9, 828.2]",1,0,0,0
4,4,set01,video_0004,00013.png,13,1_4_112,"[1085.74, 801.97, 1095.32, 830.3]",1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
623,623,set04,video_0013,01748.png,97,4_13_1637,"[1203.45, 686.41, 1268.82, 876.12]",0,0,0,0
624,624,set04,video_0013,01748.png,56,4_13_1638,"[1123.07, 706.08, 1167.34, 855.36]",0,0,1,0
625,625,set04,video_0013,01737.png,92,4_13_1636,"[1174.95, 684.36, 1242.11, 871.16]",0,0,0,0
626,626,set04,video_0013,01737.png,86,4_13_1637,"[1148.82, 691.26, 1211.68, 860.29]",0,0,0,0


In [49]:
df_val = pd.read_csv('../val_annotations.csv')
df_val

,Unnamed: 0,set,video,frame,ped_idx,ped,BBox,action,gesture,look,cross
0,0,set05,video_0001,15664.png,91,5_1_1749,"[1033.83, 720.43, 1050.6, 766.5]",1,0,0,0
1,1,set05,video_0001,15665.png,92,5_1_1749,"[1034.04, 720.36, 1050.67, 766.82]",1,0,0,0
2,2,set05,video_0001,15674.png,101,5_1_1749,"[1037.15, 722.38, 1054.36, 773.68]",1,0,0,0
3,3,set05,video_0001,15663.png,90,5_1_1749,"[1032.98, 721.49, 1049.47, 766.98]",1,0,0,0
4,4,set05,video_0001,15672.png,99,5_1_1749,"[1036.13, 721.19, 1052.68, 771.04]",1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
235,235,set06,video_0002,03599.png,236,6_2_1768,"[1693.67, 570.73, 1898.24, 1080.0]",1,0,0,0
236,236,set06,video_0002,03591.png,228,6_2_1768,"[1536.72, 617.89, 1716.84, 1080.0]",1,0,0,0
237,237,set06,video_0002,03601.png,238,6_2_1768,"[1745.74, 552.18, 1909.12, 1080.0]",1,0,0,0
238,238,set06,video_0002,03590.png,227,6_2_1768,"[1520.68, 623.09, 1699.46, 1080.0]",1,0,0,0


In [51]:
df_test = pd.read_csv('../test_annotations.csv')
df_test

,Unnamed: 0,set,video,frame,ped_idx,ped,BBox,action,gesture,look,cross
0,0,set03,video_0005,00471.png,38,3_5_384,"[1123.14, 721.37, 1144.85, 780.91]",1,0,0,0
1,1,set03,video_0005,00473.png,40,3_5_384,"[1125.81, 721.07, 1147.98, 780.37]",1,0,0,0
2,2,set03,video_0005,00477.png,44,3_5_384,"[1131.15, 720.48, 1154.24, 779.29]",1,0,0,0
3,3,set03,video_0005,00478.png,45,3_5_384,"[1132.49, 720.34, 1155.81, 779.02]",1,0,0,0
4,4,set03,video_0005,00476.png,43,3_5_384,"[1129.82, 720.63, 1152.68, 779.56]",1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
646,646,set03,video_0013,00491.png,6,3_13_807,"[491.97, 723.17, 506.29, 776.85]",1,0,0,0
647,647,set03,video_0013,00485.png,0,3_13_807,"[491.59, 722.21, 505.7, 776.8]",1,0,0,0
648,648,set03,video_0013,00497.png,12,3_13_807,"[492.45, 723.49, 508.05, 777.17]",1,0,0,0
649,649,set03,video_0013,00498.png,13,3_13_807,"[492.53, 723.55, 508.34, 777.22]",1,0,0,0
